# ResidueSelectors范围选择器

氨基酸选择器(ResidueSelector)具有十分重要的功能。它能够从蛋白质结构(Pose)中选取并生成氨基酸子集。一旦生成了这些子集，对后续建模的逻辑操作具有重大的意义，比如可以定义设计或采样的自由度（使用ResidueSelector可以将蛋白质距离内核中心5埃范围内的氨基酸选择出来，后续进行氨基酸侧链能量最小化等结构优化），也可以配合**SimpleMetrics**、**Filter**等进行蛋白质性质或参数的统计。

注: ResidueSelectors的概念比较简单也比较利于初学者理解，因此此章节学习难度较小。

### 1. ResidueSelector与vector1_bool

在PyRosetta中，定义好ResidueSelectors后，进行apply(可以理解为执行选择的过程)，我们将得到氨基酸残基的子集列表。这个列表被保存在vector1<bool>对象中，以下以具体的实例进行讲解:

In [22]:
# 导入链选择器
from pyrosetta import pose_from_pdb, init
from pyrosetta.rosetta.core.select.residue_selector import ChainSelector

init()
# 从pdb中读入生成pose对象，(肝细胞生长因子抗体PDB:6LZ9)
pose = pose_from_pdb('./data/6LZ9_H_L.pdb')

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.mac.python37.Release 2020.02+release.22ef835b4a2647af94fcd6421a85720f07eddf12 2020-01-05T17:31:56] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.python37.Release r242 2020.02+release.22ef835b4a2 22ef835b4a2647af94fcd6421a85720f07eddf12 http://www.pyrosetta.org 2020-01-05T17:31:56
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /opt/miniconda3/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=437453851 seed_offset=0 real_seed=437453851 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=437453851 RG_type=mt19937
core.import_pose.import_pose: {0} File './data/6LZ9_H_L.pdb' automatically deter

In [23]:
# 先来看看抗体的基本信息:
print(f'抗体含有的链数量:{pose.num_chains()}')
print(f'抗体含有的氨基酸数量:{pose.total_residue()}')

抗体含有的链数量:2
抗体含有的氨基酸数量:223


In [24]:
# 选择抗体的重链，PDB链号为"H":
select_heavy_chain = ChainSelector('H')
select_heavy_chain.apply(pose)

vector1_bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [25]:
# 查看1号和223号氨基酸的PDB信息:
print(pose.pdb_info().pose2pdb(1))
print(pose.pdb_info().pose2pdb(223))

2 H 
105 L 


**结果解读**</br>
知识点1: vector1_bool中被选择的氨基酸返回“1”，而没有被选择的氨基酸返回“0”</br>
知识点2: vector1_bool中是按照Pose编号进行编写的(从1开开始)，也就是说重链的编号从1 -> n, 轻链的编号从n+1 -> 223.

### 2. ResidueSelector的基本分类

氨基酸选择器按功能可分为三大类:

- 逻辑选择器
- 非构象依赖选择器
- 构象依赖选择器

以下我们将逐步来讲解在实战中，都有哪些氨基酸选择可以为我们所用。

#### 2.1 逻辑选择器
第一部分是逻辑选择器，很好理解，按照逻辑分类为Not、And、Or逻辑关系，可以将**两个**选择器进行逻辑的再次选择。
在Rosetta中，负责逻辑定义的选择器为NotResidueSelector、AndResidueSelector、OrResidueSelector。
以下做实例说明:

In [26]:
# 还是以之前读入的抗体pose为例。
# 先定义Selector:
select_heavy_chain = ChainSelector('H')
select_light_chain = ChainSelector('L')

In [27]:
# 选择重链，并生成氨基酸子集
select_heavy_chain.apply(pose)

vector1_bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [28]:
# 选择轻链，并生成氨基酸子集
select_light_chain.apply(pose)

vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [29]:
# 选择轻链**或**重链
from pyrosetta.rosetta.core.select.residue_selector import OrResidueSelector
light_or_heavy = OrResidueSelector(select_heavy_chain, select_light_chain)
light_or_heavy.apply(pose)

vector1_bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

**结果解读**:</br>
可见vector1_bool返回了所有的氨基酸，因为pose中只有重链和轻链，因此或逻辑返回了所有的氨基酸

In [30]:
# 选择重链**且**轻链
from pyrosetta.rosetta.core.select.residue_selector import AndResidueSelector
light_and_heavy = AndResidueSelector(select_heavy_chain, select_light_chain)
light_and_heavy.apply(pose)

vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

**结果解读**:</br>
可见vector1_bool的子集列表全为0，因为重链和轻链是独立的两条链，不存在”且“的关系。

In [31]:
# 非选择器:
from pyrosetta.rosetta.core.select.residue_selector import NotResidueSelector
not_heavy = NotResidueSelector(select_heavy_chain)
not_heavy.apply(pose)

vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

**结果解读**:</br>
可见vector1_bool的返回了轻链的氨基酸子集（非重链的部分）。

#### 2.2 非构象依赖的选择器
这类选择器的定义不依赖于具体的构象，仅仅依靠属性就可以定义。如链的选择、选择某号氨基酸、选择带电氨基酸等。以下将逐一介绍这些选择器，以及其使用的方法:

#### 2.2.1 ResidueIndexSelector
通过氨基酸的具体编号定义的选择器，不仅可以使用PDB编号、Pose编号，还可以指定氨基酸的范围进行选择。

In [32]:
from pyrosetta.rosetta.core.select.residue_selector import ResidueIndexSelector
# 根据具体的编号选择:
pose_index_selector = ResidueIndexSelector('1,3,5')
pose_index_selector.apply(pose)

vector1_bool[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [33]:
# 根据具体的PDB编号选择, 注意需要附带上PDB链的信息。
pdb_index_selector = ResidueIndexSelector('2H,3H,4H')
pdb_index_selector.apply(pose)

vector1_bool[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [34]:
# 根据PDB的范围进行选择。
range_selector = ResidueIndexSelector('2H-20H')
range_selector.apply(pose)

vector1_bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

#### 2.2.2 ResidueNameSelector
通过氨基酸的具体残基名定义的选择器:

In [35]:
# Resname selector; 残基名称选择器
from pyrosetta.rosetta.core.select.residue_selector import *
resname_selector = ResidueNameSelector('PHE')
resname_selector.apply(pose)

vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [36]:
# Resname selector; 残基名称选择器, 根据多个残基名进行选择:
resname_selector = ResidueNameSelector('PHE,ASN')
resname_selector.apply(pose)

vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [37]:
# Resname selector; 选择带修饰的氨基酸残基
resname_selector = ResidueNameSelector('CYS')
resname_selector.apply(pose)

vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [38]:
print(pose.residue(21))

Residue 21: CYS:disulfide (CYS, C):
Base: CYS
 Properties: POLYMER PROTEIN CANONICAL_AA SC_ORBITALS METALBINDING DISULFIDE_BONDED ALPHA_AA L_AA
 Variant types: DISULFIDE
 Main-chain atoms:  N    CA   C  
 Backbone atoms:    N    CA   C    O    H    HA 
 Side-chain atoms:  CB   SG  1HB  2HB 
Atom Coordinates:
   N  : 41.714, 50.854, 39.059
   CA : 40.257, 50.826, 39.06
   C  : 39.723, 49.899, 37.976
   O  : 39.898, 50.156, 36.784
   CB : 39.691, 52.23, 38.847
   SG : 37.891, 52.332, 38.998
   H  : 42.196, 51.355, 38.325
   HA : 39.919, 50.458, 40.029
  1HB : 40.13, 52.913, 39.575
  2HB : 39.967, 52.587, 37.855
Mirrored relative to coordinates in ResidueType: FALSE



**结果解读**</br>
选择带二硫键的氨基酸时，使用CYS残基名并没有正确选择到对应的氨基酸，因为在Rosetta中，形成二硫键的半胱氨酸名为CYS:
CYS:disulfide, 接下来我们尝试换个名字进行选择.

In [39]:
resname_selector = ResidueNameSelector('CYS:disulfide')
resname_selector.apply(pose)

vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

**结果解读**</br>
现在可以正确选择到对应的二硫键氨基酸子集了！

#### 2.2.2 ResiduePropertySelector
通过氨基酸的性质进行定义的选择器，比如可以选择带正电的氨基酸、带负电的氨基酸、疏水氨基酸、含有芳香环的氨基酸等等。</br>
氨基酸的性质分类可详见: https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/ResidueSelectors/ResiduePropertySelector

以下以实例进行说明:

In [40]:
from pyrosetta.rosetta.core.select.residue_selector import ResiduePropertySelector
from pyrosetta.rosetta.core.chemical import ResidueProperty

# example1: 使用单一属性选择，选择所有的极性氨基酸
prop_selector = ResiduePropertySelector(ResidueProperty.POLAR)
prop_selector.apply(pose)

vector1_bool[0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1]

In [41]:
from pyrosetta.rosetta.core.select.residue_selector import basic_selection_logic
# example2: 使用多属性进行选择
prop_selector = ResiduePropertySelector()
prop_selector.add_property(ResidueProperty.POLAR)  # 极性氨基酸
prop_selector.add_property(ResidueProperty.CHARGED) # 带电的氨基酸
prop_selector.set_selection_logic(basic_selection_logic.or_logic)
prop_selector.apply(pose)

vector1_bool[0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1]

**结果解读**</br>
注意在使用多属性选择时，应当注意选择的逻辑性，可通过set_selection_logic进行设置，默认为AND逻辑。如果设置为OR逻辑需要使用basic_selection_logic.or_logic.

#### 2.2.3 RandomResidueSelector
从某个氨基酸子集中随机地选取N个氨基酸定义的选择器。

In [42]:
from pyrosetta.rosetta.core.select.residue_selector import RandomResidueSelector

# 比如从重链中随机选择出5个氨基酸: 多运行几次，就可发现返回的编号是不一样的。
random_selector = RandomResidueSelector(select_heavy_chain, 30)
random_selector.apply(pose)

core.select.residue_selector.RandomResidueSelector: {0} Selected residues: 19 56 113 105 82 44 98 24 43 23 22 39 38 29 15 10 26 5 34 37 110 65 49 57 119 80 32 3 73 11


vector1_bool[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

#### 2.2.4 Antibody相关的选择器
也有一些选择器是根据分子类型开发的，比如在RosettaAntibody模块中，就有特定选择CDR氨基酸的选择器。比如CDRResidueSelector、AntibodyRegionSelector等。

##### 1. CDRResidueSelector

In [43]:
from pyrosetta.rosetta.protocols.antibody import CDRNameEnum
from rosetta.protocols.antibody.residue_selector import CDRResidueSelector

In [44]:
init('-input_ab_scheme Chothia') # 根据抗体的不同编号进行选择初始化

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.mac.python37.Release 2020.02+release.22ef835b4a2647af94fcd6421a85720f07eddf12 2020-01-05T17:31:56] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.python37.Release r242 2020.02+release.22ef835b4a2 22ef835b4a2647af94fcd6421a85720f07eddf12 http://www.pyrosetta.org 2020-01-05T17:31:56
core.init: {0} command: PyRosetta -input_ab_scheme Chothia -database /opt/miniconda3/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-1409532468 seed_offset=0 real_seed=-1409532468 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=-1409532468 RG_type=mt19937


In [45]:
# example1: 选择单个CDR.
cdr_selector = CDRResidueSelector()
cdr_selector.set_cdr(CDRNameEnum.h1)  #h1,h2,h3,l1,l2,l3
cdr_selector.apply(pose)

basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} TRDGGLLFAYYAMDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 9 Omega: TTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H3
protocols.antibody.cluster.C

vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [46]:
# example2: 选择多个CDR
from pyrosetta.rosetta.utility import vector1_protocols_antibody_CDRNameEnum

# 设定cdr list;
cdrs_list = vector1_protocols_antibody_CDRNameEnum()
cdrs_list.append(CDRNameEnum.h1) # 添加cdr-h1
cdrs_list.append(CDRNameEnum.h2) # 添加cdr-h2
cdrs_list.append(CDRNameEnum.h3) # 添加cdr-h3

cdr_selector = CDRResidueSelector()
cdr_selector.set_cdrs(cdrs_list)
cdr_selector.apply(pose)

basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} TRDGGLLFAYYAMDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 9 Omega: TTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H3
protocols.antibody.cluster.C

vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

##### 2. AntibodyRegionSelector
除了选择CDR，还可以通过抗体的区域选择器选择Framework区、抗体antigen等区域。
使用AntibodyRegionSelector选择器需要额外定义好AntibodyRegionEnum:
- antigen_region
- cdr_region
- framework_region

In [47]:
# 抗体区域选择器:
from pyrosetta.rosetta.protocols.antibody import AntibodyRegionEnum
from pyrosetta.rosetta.protocols.antibody.residue_selector import AntibodyRegionSelector
ab_region_selector = AntibodyRegionSelector()
ab_region_selector.set_region(AntibodyRegionEnum.cdr_region)
ab_region_selector.apply(pose)

# 读者也可以尝试设置以下的一些选项试试。
ab_region_selector.set_region(framework_region)
ab_region_selector.set_region(antigen_region)

basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} TRDGGLLFAYYAMDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 9 Omega: TTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H3
protocols.antibody.cluster.C

#### 2.2.5 糖分子相关的选择器
糖分子的建模也是近期Rosetta兼容的对象之一，现在有几个专门地选择器GlycanResidueSelector、GlycanLayerSelector、RandomGlycanFoliageSelector。

In [48]:
#略

### 2.3 构象依赖的选择器
顾名思义，这类选择器与分子结构的具体构象有关，具体地由二面角、二级结构、氢键、邻居分子数量、相互作用界面、对称性等几个层次去进行定义。

#### 2.3.1 InterGroupInterfaceByVectorSelector
该选择器使用氨基酸Cα-Cβ向量的长度以及角度来搜索两个刚体之间相互接触的氨基酸, 其中nearby_atom_cut和vector_dist_cut是关键的距离参数，距离越大，选择的相互作用界面越大。默认为6和8埃。

In [54]:
from pyrosetta.rosetta.core.select.residue_selector import InterGroupInterfaceByVectorSelector
interface_seletor = InterGroupInterfaceByVectorSelector(select_light_chain, select_heavy_chain)
interface_seletor.nearby_atom_cut(6)
interface_seletor.vector_dist_cut(8)
interface_seletor.apply(pose)

vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0]

#### 2.3.2 LayerSelector
Layer是Rosetta中的一个重要概念，将蛋白质分为表面层(surface)、交界层(boundary)以及内核层(Core)。众所周知，蛋白质的不同层的氨基酸是具有偏好性分布的，比如在内核层由许多的非极性氨基酸组成的疏水核心，而在表面层分布大多为极性氨基酸，能与溶液环境中的水分子相互作用。LayerSelector是十分强大的工具，可以根据set_layers()进行组合选择。set_layers()的三个布尔值按顺序分别代表选择Core/boundary/Surface。另外该选择器有两种判别模式（只能选其一），分别是基于SASA的方法和基于邻居数量的方法。

In [62]:
from pyrosetta.rosetta.core.select.residue_selector import LayerSelector
layer_selector = LayerSelector()

# 定义需要选定的层:
layer_selector.set_layers(True, False, False) # 只选择内核的设置方法
layer_selector.set_layers(False, True, False) # 只选择边界层的设置方法
layer_selector.set_layers(False, False, True) # 只选择表面层的设置方法 

# neighbor-based(选1，根据set_use_sc_neighbors)
layer_selector.set_use_sc_neighbors(False) # 使用Sidechain neighbor算法，否则使用SASA-based算法，确定内核残基。
layer_selector.set_cutoffs(5.2, 2.0) # 内核,表层邻居数量截断设置

# neighbor-based(选1，根据set_use_sc_neighbors)
layer_selector.set_use_sc_neighbors(True)
layer_selector.set_cutoffs(5.2, 2.0) # 内核,表层邻居数量截断设置

core.select.residue_selector.LayerSelector: {0} Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: {0} Set cutoffs for core and surface to 5.2 and 2, respectively, in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting core=true boundary=false surface=false in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting core=false boundary=true surface=false in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting core=false boundary=false surface=true in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: {0} Set cutoffs for core and surface to 5.2 and 2, respectively, in LayerSelector.


In [65]:
# example1: 选择内核层(SASA法)
layer_selector = LayerSelector()
layer_selector.set_layers(True, False, False) # 只选择内核的设置方法
layer_selector.set_use_sc_neighbors(False) # 使用Sidechain neighbor算法，否则使用SASA-based算法，确定内核残基。
layer_selector.set_cutoffs(20.0, 40.0) # 内核,表层截断半径设置
layer_selector.set_ball_radius(2.0) # 传统值为1.4
layer_selector.apply(pose)

core.select.residue_selector.LayerSelector: {0} Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: {0} Set cutoffs for core and surface to 5.2 and 2, respectively, in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting core=true boundary=false surface=false in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting LayerSelector to use rolling ball-based occlusion to determine burial.
core.select.residue_selector.LayerSelector: {0} Set cutoffs for core and surface to 20 and 40, respectively, in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting radius for rolling ball algorithm to 2 in LayerSelector.  (Note that this will have no effect if the sidechain neighbors method is used.)


vector1_bool[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0]

In [66]:
# example2: 选择表面层(侧链法)
layer_selector = LayerSelector()
layer_selector.set_layers(False, False, True) # 只选择内核的设置方法
layer_selector.set_use_sc_neighbors(True) # 使用Sidechain neighbor算法，否则使用SASA-based算法，确定内核残基。
layer_selector.set_cutoffs(5.2, 2.0) # 内核,表层邻居数量截断设置
layer_selector.apply(pose)

core.select.residue_selector.LayerSelector: {0} Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: {0} Set cutoffs for core and surface to 5.2 and 2, respectively, in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting core=false boundary=false surface=true in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: {0} Set cutoffs for core and surface to 5.2 and 2, respectively, in LayerSelector.


vector1_bool[0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]

#### 2.3.3 二面角相关选择器
在Rosetta中，二面角作为描述蛋白骨架结构最重要的参数，当然少不了与二面角数据有关的选择器。此处主要介绍BinSelector、PhiSelector。

##### 1. BinSelector

ABEGO系统在Rosetta中也是比较重要的概念，是从Ramachandran-plot的分布区间进行定义的离散模型, 坐标轴分别定义了phi/psi角，用于描述蛋白局部骨架结构的构象。
ABEGO的每个字母代表一个区间, 这些区间的分布是有二级结构的偏向性的:
- A: 二面角分布多见于右手性的α螺旋
- B: 二面角分布多见于右手性的β折叠
- E: 二面角分布多见于左手性的β折叠（罕见）
- G: 二面角分布多见于左手性的螺旋（罕见）
- O: 反式的omega分布，多见于反式脯氨酸

L-氨基酸的二面角的分布频率可见下图（D-氨基酸的分布频率为镜像）:

![](./img/Coarse-grained-representation-of-the-Ramachandran-plot-based-on-ABEGO-torsion-bins-ABEGO.jpg)

In [73]:
# 将满足phi、psi落在某个区间骨架二面角的残基选出。
from pyrosetta.rosetta.core.select.residue_selector import BinSelector
bin_selector = BinSelector()
bin_selector.set_bin_name('A') # 选择落在A区域的氨基酸
bin_selector.set_bin_params_file_name('ABEGO')
bin_selector.initialize_and_check() # 必须先启动
bin_selector.apply(pose)

basic.io.database: {0} Database file opened: protocol_data/generalizedKIC/bin_params/ABEGO.bin_params
core.scoring.bin_transitions.BinTransitionCalculator: {0} Opened file protocol_data/generalizedKIC/bin_params/ABEGO.bin_params for read.
core.scoring.bin_transitions.BinTransitionCalculator: {0} Finished read of protocol_data/generalizedKIC/bin_params/ABEGO.bin_params.
core.select.residue_selector.BinSelector: {0} Loaded bin parameters file "ABEGO" and set bin to select to "A".  Only selecting alpha-amino acids.


vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

##### 2. PhiSelector
根据Ramachandran空间某一坐标轴的正负值，进行选择氨基酸。</br>
该选择器**有特定的应用场景**，比如在环肽设计中，通常会混合D-/L-氨基酸。那这些氨基酸的ABGEO分布式对称的，一般用甘氨酸作为出发的氨基酸进行骨架采样，当骨架phi为正值时，这些位置就可以设计为D型氨基酸，而phi为负值时，设计为L型氨基酸。这种设计方案显得更有"物理"意义。

In [77]:
from pyrosetta.rosetta.core.select.residue_selector import PhiSelector
phi_selector = PhiSelector()
phi_selector.set_select_positive_phi(True)  # True=select +phi
phi_selector.set_ignore_unconnected_upper(True)
phi_selector.apply(pose)

vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

##### 3. PsiSelector
没有这个选择器...有兴趣的读者可以尝试自己写一个。

#### 2.3.4 邻居相关的选择器
邻居的概念应用十分广泛，比如我想选择酶活中心周围8埃距离范围内的所有氨基酸等，该选择器对区域性设计时十分有用！

##### 1. NeighborhoodResidueSelector
有两种用法，第一种选择半径范围内所有的氨基酸，第二种为选择**邻近范围内**的氨基酸

In [82]:
# 比如我想选择1号氨基酸范围内10埃所有的氨基酸(包括1号氨基酸):
from pyrosetta.rosetta.core.select.residue_selector import NeighborhoodResidueSelector, ResidueIndexSelector
residue1_selector = ResidueIndexSelector('1')
nbr_selector = NeighborhoodResidueSelector(residue1_selector, 10.0, True)  # True 代表包括1号氨基酸。
nbr_selector.apply(pose)

core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] ################ Cloning pose and building neighbor graph ################
core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] Ensure that pose is either scored or has update_residue_neighbors() called
core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] before using NeighborhoodResidueSelector for maximum performance!
core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] ##########################################################################


vector1_bool[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [83]:
# 比如我想选择1号氨基酸范围内10埃所有的氨基酸(不包括1号氨基酸):
nbr_selector = NeighborhoodResidueSelector(residue1_selector, 10.0, False)  # True 代表包括1号氨基酸。
nbr_selector.apply(pose)

core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] ################ Cloning pose and building neighbor graph ################
core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] Ensure that pose is either scored or has update_residue_neighbors() called
core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] before using NeighborhoodResidueSelector for maximum performance!
core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] ##########################################################################


vector1_bool[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

##### 2. NumNeighborsSelector
不同于NeighborhoodResidueSelector，NumNeighborsSelector只会将周围邻居氨基酸超过设定阈值部分的氨基酸位点选出，对于选择氨基酸密度高的区域比较有用。

In [90]:
from pyrosetta.rosetta.core.select.residue_selector import NumNeighborsSelector
nn_selector = NumNeighborsSelector(15, 10.0)  # 两个参数分别是邻居数量阈值、距离半径阈值
nn_selector.count_water(True)  # 甚至还可以考虑水分子的数量.
nn_selector.apply(pose)

vector1_bool[0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]

##### 3. CloseContactResidueSelector

#### 2.3.5 一、二级结构相关的选择器
基于一、二级结构的选择器种类丰富:
- PrimarySequenceNeighborhoodSelector
- SSElementSelector
- SecondaryStructureSelector
- PairedSheetResidueSelector

##### PrimarySequenceNeighborhoodSelector

In [ ]:
TaskSelector